# Test and Train ANNs

This notebook will train and test an MLP classifier and 9 MLP regressors (1 for each drummer). The classifier will classifer what drummer is playing based on microtiming and velocity, and the regressors will predict microtiming histogram values based on velocity histogram values for each drummer. The outputs will be a joblib file for the classifier pipeline and CSV files containing histogram bin percentages for velocity and microtiming for drummers 3, 7, 9 (which were shown to have the best results).

In [1]:
#importing packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import os
from mido import MidiFile
import sklearn
import joblib
from classes import Note
from classes import File
from classes import Drummer
from functions import extractMidiData

# Extract MIDI Data From Files

In [2]:
drummerNameToInt = {
    "drummer1": 1,
    "drummer3": 3,
    "drummer4": 4,
    "drummer5": 5,
    "drummer6": 6,
    "drummer7": 7,
    "drummer8": 8,
    "drummer9": 9,
    "drummer10": 10
}

In [3]:
drummers = os.listdir('./extendedGroove')

drummersList = []

# Keeping track of the number of files available, not all will be included all the time because
# I will be focusing on files that have snare, hi-hats, and bass drum notes 
maxNumberOfFiles = 0

for drummer in drummers:
    if drummer.find('drummer') != -1:
        filesList = []
        sessions = os.listdir('./extendedGroove/' + drummer)
        for session in sessions:
            if session != '.DS_Store' and session != 'Icon\r' and session != "eval_session":
                files = os.listdir('./extendedGroove/' + drummer + '/' + session)
                for file in files:
                    if file.endswith('.midi'):
                        maxNumberOfFiles += 1
                        filePath = './extendedGroove/' + drummer + "/" + session + "/"  + file
                        filesList.append(extractMidiData(filePath))
        
        drummersList.append(Drummer(drummer, drummerNameToInt[drummer], filesList))

# Train and Run Classifier

In [4]:
numberOfFeatures = 78
features = np.zeros((maxNumberOfFiles, numberOfFeatures)) 
labels = np.zeros((maxNumberOfFiles))
fileIndex = 0
for i, drummer in enumerate(drummersList):
    for file in drummer.files:
        velocities, microtimings = file.extractFeatures()
        if not isinstance(velocities, int):
            features[fileIndex, :] = np.concatenate((velocities, microtimings))
            labels[fileIndex] = drummer.num
            fileIndex += 1
            
features = np.resize(features, (fileIndex, numberOfFeatures))
labels = np.resize(labels, (fileIndex))

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

classifierPipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', MLPClassifier(hidden_layer_sizes=(10,5), max_iter=5000, activation='relu'))
        ])

# Validate Classfier with 10 Cross-Folds

In [6]:
scores = sklearn.model_selection.cross_validate(classifierPipe, features, labels, cv=10,scoring=('f1_macro', 'accuracy'),return_train_score=True)

print(scores,'\n')
print('Accuracy mean and variance', np.mean(scores['test_accuracy']),np.var(scores['test_accuracy']),'\n')
print('F1 macro mean and variance', np.mean(scores['test_f1_macro']),np.var(scores['test_f1_macro']),'\n')

{'fit_time': array([13.90044188,  8.06874895, 14.06949306,  8.08291674, 10.98228788,
       13.21937513, 13.37972116,  8.65519094, 14.44682884,  7.98129916]), 'score_time': array([0.0234282 , 0.00427508, 0.00454617, 0.00426602, 0.00452709,
       0.00459099, 0.00481105, 0.00426292, 0.00436592, 0.00434899]), 'test_f1_macro': array([0.80275922, 0.99783285, 0.987829  , 0.99832975, 0.99302094,
       0.96144925, 0.96064021, 0.99839471, 0.99394161, 0.99866553]), 'train_f1_macro': array([0.99942871, 0.99991559, 0.9992812 , 0.99926167, 0.9987962 ,
       0.98031404, 0.98243608, 1.        , 0.99711083, 0.99979304]), 'test_accuracy': array([0.77791667, 0.99833333, 0.99375   , 0.99916632, 0.99749896,
       0.99249687, 0.99249687, 0.9979158 , 0.99833264, 0.99874948]), 'train_accuracy': array([0.99986107, 0.99990738, 0.99981475, 0.99990738, 0.99962953,
       0.99680467, 0.99689729, 1.        , 0.9994906 , 0.99986107])} 

Accuracy mean and variance 0.974665694039183 0.004307280847466153 

F1 macr

# Train Model to Save for Later

In [7]:
from sklearn.model_selection import train_test_split

feat_train, feat_test, lab_train, lab_test = train_test_split(features, labels, test_size=0.2)
classifierPipe.fit(feat_train, lab_train)

joblib.dump(classifierPipe, "classifierPipe.pkl")

['classifierPipe.pkl']

# Train and Run Regressor

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

numberOfFeatures = 48
numberOfTargets = 30

for i, drummer in enumerate(drummersList):
    features = np.zeros((maxNumberOfFiles, numberOfFeatures)) 
    targets = np.zeros((maxNumberOfFiles, numberOfTargets))
    fileIndex = 0
    for file in drummer.files:
        velocities, microtimings = file.extractFeatures()
        if not isinstance(velocities, int):
            features[fileIndex, :] = velocities
            targets[fileIndex, :] = microtimings
            fileIndex += 1

    features = np.resize(features, (fileIndex, numberOfFeatures))
    targets = np.resize(targets, (fileIndex, numberOfTargets))
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', MLPRegressor(hidden_layer_sizes=(200,100,50), max_iter=5000, tol=0.0000001, activation='tanh'))
        ])
    
    scores = sklearn.model_selection.cross_validate(pipe, features, targets, cv=10, scoring=('neg_mean_squared_error', 'neg_median_absolute_error', 'r2', 'explained_variance'), return_train_score=True)
    print(drummer.name)
    print("number of files:" + str(fileIndex))
    print('Mean squared error mean and variance', np.mean(scores['test_neg_mean_squared_error']),np.var(scores['test_neg_mean_squared_error']),'\n')
    print('Median absolute error mean and variance', np.mean(scores['test_neg_median_absolute_error']),np.var(scores['test_neg_median_absolute_error']),'\n')
    print('R2 mean and variance', np.mean(scores['test_r2']),np.var(scores['test_r2']),'\n')
    print('Explained variance mean and variance', np.mean(scores['test_explained_variance']),np.var(scores['test_explained_variance']),'\n')

drummer8
number of files:1143
Mean squared error mean and variance -0.00022017367811193986 1.301752089296572e-07 

Median absolute error mean and variance -0.002800583341305232 7.766709283642221e-07 

R2 mean and variance 0.7247221722002608 0.168542259999715 

Explained variance mean and variance 0.7318973915267653 0.15864751211398426 

drummer6
number of files:584
Mean squared error mean and variance -0.0008559179585538737 3.018480738543205e-06 

Median absolute error mean and variance -0.001561138319640196 4.437349561687285e-07 

R2 mean and variance -4.235696813446863 198.1124226421635 

Explained variance mean and variance -4.063860735138258 185.09191008773485 

drummer1
number of files:9833
Mean squared error mean and variance -0.00304426386958469 7.709636983667285e-05 

Median absolute error mean and variance -0.011712359303005977 0.0003816180185746883 

R2 mean and variance 0.5210214428260463 2.0137105616131157 

Explained variance mean and variance 0.6263772288284811 1.21786194

# Generate MIDI 

Through repeated testing, drummer7, drummer9, and drummer3 were found to have the best, most consistent results (high r2 scores). Therefore, only these three drummers will have CSV files generated for them to use to generate MIDI in the midi.py python file

In [12]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

numberOfFeatures = 48
numberOfTargets = 30

for i, drummer in enumerate(drummersList):
    # Extract features the same way as before
    if(drummer.name == "drummer3") or (drummer.name == "drummer9") or (drummer.name == "drummer7"):
        features = np.zeros((maxNumberOfFiles, numberOfFeatures)) 
        targets = np.zeros((maxNumberOfFiles, numberOfTargets))
        fileIndex = 0
        for file in drummer.files:
            velocities, microtimings = file.extractFeatures()
            if not isinstance(velocities, int):
                features[fileIndex, :] = velocities
                targets[fileIndex, :] = microtimings
                fileIndex += 1

        features = np.resize(features, (fileIndex, numberOfFeatures))
        targets = np.resize(targets, (fileIndex, numberOfTargets))
        feat_train, feat_test, tar_train, tar_test = train_test_split(features, targets, test_size=0.2)
        
        # train and test regressor
        pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('regressor', MLPRegressor(hidden_layer_sizes=(200,100,50), max_iter=5000, tol=0.0000001, activation='tanh'))
            ])

        drummer.setRegressorPipe(pipe)
        drummer.regressorPipe.fit(feat_train, tar_train)
        tar_pred = drummer.regressorPipe.predict(feat_test)
        
        # Write the first microtiming and velocity values to CSV files to use in midi.py
        snareMtCol, bassDrumMtCol, hihatMtCol = np.split(tar_pred[0], 3)
        dfDict = {
            'snareMicrotiming': snareMtCol,
            'bassDrumMicrotiming': bassDrumMtCol,
            'hihatMicrotiming': hihatMtCol,
        }
        df = pd.DataFrame(dfDict)
        df.to_csv(drummer.name + 'Microtiming.csv')
        
        snareVelocityCol, bassDrumVelocityCol, hihatVelocityCol = np.split(feat_test[0, :48], 3)
        dfDict = {
            'snareVelocity': snareVelocityCol,
            'bassDrumVelocity': bassDrumVelocityCol,
            'hihatVelocity': hihatVelocityCol,
        }
        
        df = pd.DataFrame(dfDict)
        df.to_csv(drummer.name + 'Velocity.csv')
        
        # Print results
        print(drummer.name)
        print("number of files:" + str(fileIndex))
        
        print('Mean squared error: %.4f'% sklearn.metrics.mean_squared_error(tar_test, tar_pred))
        print('Mean absolute error: %.4f'% sklearn.metrics.mean_absolute_error(tar_test, tar_pred))
        print('Coefficient of determination (R2 score): %.4f'% sklearn.metrics.r2_score(tar_test, tar_pred))
        print('Explained variance score: %.4f'% sklearn.metrics.explained_variance_score(tar_test, tar_pred))
        print('R2 score on individual targets',sklearn.metrics.r2_score(tar_test, tar_pred, multioutput='raw_values') )

drummer7
number of files:7337
Mean squared error: 0.0002
Mean absolute error: 0.0068
Coefficient of determination (R2 score): 0.9871
Explained variance score: 0.9873
R2 score on individual targets [0.98742443 0.96268569 0.98167019 0.98838232 0.99134503 0.98886871
 0.9897513  0.98831038 0.99388701 0.9894396  0.98908558 0.9911004
 0.99022776 0.96518424 0.98604452 0.9977991  0.99236185 0.98754653
 0.98462413 0.990326   0.98766936 0.98812296 0.99319373 0.99089754
 0.97782143 0.97978438 0.98549155 0.99139512 0.99254514 0.99004696]
drummer9
number of files:1104
Mean squared error: 0.0000
Mean absolute error: 0.0024
Coefficient of determination (R2 score): 0.9953
Explained variance score: 0.9954
R2 score on individual targets [0.99066513 0.99184607 0.99446545 0.99223017 0.99616538 0.9919786
 0.99633957 0.99575436 0.99758029 0.99509047 0.99612671 0.99282624
 0.99478749 0.98990414 0.99459213 0.99904942 0.98912187 0.98976317
 0.99735075 0.99633459 0.9948145  0.99883011 0.99510753 0.99871922
 0.9